In [1]:
#kein undersampling sampling
%store -r hash_texts
%store -r hash_labels

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk


In [3]:
#######clean#######

In [4]:
from num2words import num2words
#convert everything to lowercase
hash_texts = [x.lower() for x in hash_texts]
#exchange numbers with their string representation
#make numbers to text
temp = []
without_num = []
for post in hash_texts:
    for word in post.split(" "):
        if word.isdigit():
            #print(word)
            word = num2words(word, lang="de")
        temp.append(word)
    without_num.append((' '.join(temp)))
    temp = []
hash_texts = without_num

In [5]:
#######feature extraction#########

In [6]:
#emoji feature
emojis = ["😷","😫","🤮", "😣", "😕", "😍", "😞", "😢", "😂", "🥰", "😭",
          "😱", "😒", "😔","😕", "😖", "😫", "😤", "😠", "😡", "🤬","😪"]
from copy import deepcopy
contains_emotional_emojis = deepcopy(hash_texts) #FEATURE
#print(contains_emotional_emojis)
for n, post in enumerate(contains_emotional_emojis):
    for e in emojis:
        if e in post:
            #print(post)
            contains_emotional_emojis[n] = [1]
for n,i in enumerate(contains_emotional_emojis):
    if not i == [1]:
        contains_emotional_emojis[n] = [0] 

In [7]:
#without deleting stopwords
#ngram feature matrices

#unigram
uni_vectorizer = CountVectorizer(ngram_range=(1,1))
unigram_feature_vector = uni_vectorizer.fit_transform(hash_texts).toarray() 

#uni and bigram
uni_bi_vectorizer = CountVectorizer(ngram_range=(1,2))
unigram_bi_feature_vector = uni_bi_vectorizer.fit_transform(hash_texts).toarray() 

#bigram
bi_vectorizer = CountVectorizer(ngram_range=(2,2))
bigram_feature_vector = bi_vectorizer.fit_transform(hash_texts).toarray()


#trigram
tri_vectorizer = CountVectorizer(ngram_range=(3,3))
trigram_feature_vector = tri_vectorizer.fit_transform(hash_texts).toarray() 

In [8]:
#tfidf vectorizer ohne stopwords zu löschen
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(hash_texts).toarray() #feature matrix

In [9]:
#numerical features
#average length of post, average sentence length, average number of punctuation
#punctuation weg lassen weil nicht wirklich aussagekräftig


import numpy
import string
from nltk import tokenize

post_length = [] #feature vector
counter = 0
number_of_punct = [] #feature vector
sentence_lengths = [] 
tokenized_posts = []
for text in hash_texts:
    post_length.append([len(text.split())])
    tokenized_posts.append(tokenize.sent_tokenize(text))
    for char in text:
        if char in string.punctuation:
            counter+=1
    number_of_punct.append([counter])
    counter = 0
sent = []
for post in tokenized_posts:
    for sentence in post:
        sent.append(len(sentence.split()))
    sentence_lengths.append(sent)
    sent = []
#get average
average_sent_len = [] #feature vector
for post in sentence_lengths:
    average_sent_len.append([sum(post)/len(post)])    
print(len(sentence_lengths)) #why 2169?? need:1269 need one number, maybe the average sentence length?
#print(average_sent_len)
#print(sentence_lengths)

328


In [10]:
#POS features
import spacy
nlp = spacy.load('de_core_news_md')

#get POStags via spacy
spacy_obj = []
post_as_pos_tags = []
as_pos = [] #this
for text in hash_texts:
    spacy_obj.append(nlp(text))
for text in spacy_obj:
    for token in text:
        post_as_pos_tags.append(token.tag_)
    as_pos.append(' '.join(post_as_pos_tags))
    post_as_pos_tags = []

#neue POS-features
ADV_count = []
PPER_count = []
comma_count = []
NE_count = []
NN_count = []
ADJD_count = []
for post in as_pos:
    ADV_count.append([post.count("ADV")])
    PPER_count.append([post.count("PPER")])
    comma_count.append([post.count("$,")])
    NE_count.append([post.count("NE")])
    NN_count.append([post.count("NN")])
    ADJD_count.append([post.count("ADJD")])
    
print(len(ADJD_count))

#EXPERIMENTS
#get posts as sequence of pos tags
#as_pos = sequence of pos tags
#make sequences into count vectorizers
pos_vec_uni = CountVectorizer(ngram_range=(1,1))
pos_uni_count_vec = pos_vec_uni.fit_transform(as_pos).toarray() #uni feature vector
pos_vec_bi = CountVectorizer(ngram_range=(2,2))
pos_bi_count_vec = pos_vec_bi.fit_transform(as_pos).toarray() #bi feature vector
#make theminto tfidf vectorizers
tfidf_vectorizer_pos = TfidfVectorizer()
tfidf_pos = tfidf_vectorizer_pos.fit_transform(as_pos) #tfidf pos feature matrix hat das iwelche benefits??

328


In [11]:
#document vectors as features

spacy_posts = []
vector_posts = []
temp_post = []
for post in hash_texts:
    spacy_posts.append(nlp(post))
for post in spacy_posts:
    for word in post:
        temp_post.append(word.vector)
    vector_posts.append(temp_post)
    temp_post = []
    
mean_vector_posts = []
for post_list in vector_posts:
    #print(len(post))
    mean_vector_posts.append([sum(post_list)/len(post_list)])
    
word_vector_feature = numpy.concatenate(mean_vector_posts, axis=0)
print(word_vector_feature.shape)   


(328, 300)


In [12]:
#wordvectors as features

#make all to numpy arrays
np_posts = []
for p in vector_posts:
    np_posts.append(numpy.array(p))
       
#get matrix
padded_arrays = []
for p in np_posts:
       padded_arrays.append(numpy.pad(p, ((0, (55 - p.shape[0])), (0, 0)), 'constant', constant_values=0))
print(len(padded_arrays))



np_pad_arr = numpy.array(padded_arrays)
np_pad_arr.shape #shape (97, 48, 300)

nodoc = numpy.reshape (np_pad_arr, (328, -1)) #FEATURE VECTOR FOR WORDVECTORS
print("from ", np_pad_arr.shape, " to ", nodoc.shape)


328
from  (328, 55, 300)  to  (328, 16500)


In [13]:
###################combine feature-vectors:##########################

In [14]:
#combine feature-vectors:

#uni-grams
#unigram_feature_vector

#bigrams
#bigram_feature_vector

#trigrams
#trigram_feature_vector

#unigrams and bigrams
#unigram_bigram_feature_vector

#tfidf

#alle POS-features
pos = numpy.append(ADV_count, comma_count, 1)
pos = numpy.append(pos, NE_count, 1)
pos = numpy.append(pos, NN_count, 1)
pos = numpy.append(pos, ADJD_count, 1)
pos = numpy.append(pos, PPER_count, 1)

#numerical features
#sentence_length
post_sent = numpy.append(post_length, average_sent_len, 1)
post_sent_punct = numpy.append(post_sent, number_of_punct, 1)

#documents vectors
#word_vector_feature

#wordvectors
#nodoc

#emoji feature
#contains_emotional_emojis

# 3) uni-grams, bigrams, trigrams
uni_bi_tri_feat_vec = numpy.append(unigram_feature_vector, bigram_feature_vector, 1)
uni_bi_tri_feat_vec = numpy.append(uni_bi_tri_feat_vec,trigram_feature_vector, 1)

# 4) tfidf and bigrams
tfidf_bigram = numpy.append(tfidf, bigram_feature_vector, 1)

# 5) unigrams and POS features
uni_pos_new = numpy.append(unigram_feature_vector, pos, 1)

# 6) unigrams and numerical features
uni_num = numpy.append(unigram_feature_vector, post_sent_punct, 1)

# 7) unigrams and POS tags and numerical features
uni_num_pos = numpy.append(uni_num, pos, 1)

# 8) Bigrams and POS
bi_pos_new = numpy.append(bigram_feature_vector, pos, 1)

# 9) bigrams and numerical features
bi_num = numpy.append(bigram_feature_vector, post_sent_punct, 1)

# 10) bigrams and POS tags and numerical features
bi_num_pos = numpy.append(bi_num, pos, 1)

# 11) tfidf and POS
tfidf_pos_new = numpy.append(tfidf, pos, 1)

# 12) tfidf and numerical features
tfidf_num = numpy.append(tfidf, post_sent_punct, 1)

# 13) tfidf and POS tags and numerical features
tfidf_num_pos = numpy.append(tfidf_num, pos, 1)

# 14) wordvec and POS
wordvec_pos = numpy.append(nodoc, pos, 1)

# 15) wordvec and numerical features
wordvec_num = numpy.append(nodoc, post_sent_punct, 1)

# 16) wordvec and POS tags and numerical features
wordvec_num_pos = numpy.append(wordvec_num, pos, 1)

# 17) docvec and POS
docvec_pos = numpy.append(word_vector_feature, pos, 1)

# 18) docvec and numerical features
docvec_num = numpy.append(word_vector_feature, post_sent_punct, 1)

# 19) docvec and POS tags and numerical features
docvec_num_pos = numpy.append(docvec_num, pos, 1)


# 20)POS and numerical features
pos_post_sent_len = numpy.append(pos, post_sent, 1)
pos_num = numpy.append(post_sent_punct, pos, 1)

# 21) alle
all_features = numpy.append(unigram_feature_vector, bigram_feature_vector, 1)
all_features = numpy.append(all_features, trigram_feature_vector , 1)
all_features = numpy.append(all_features, tfidf, 1)
all_features = numpy.append(all_features,post_sent_punct, 1)
all_features = numpy.append(all_features, pos, 1)
all_features = numpy.append(all_features,nodoc, 1)
all_features = numpy.append(all_features,word_vector_feature, 1)
all_features = numpy.append(all_features,contains_emotional_emojis, 1)

# 22) hash unigram
emot_uni = numpy.append(unigram_feature_vector, contains_emotional_emojis, 1)
# 23) hash bigram
emot_bi = numpy.append(bigram_feature_vector,contains_emotional_emojis, 1)
# 24) hash tfidf
emot_tfidf = numpy.append(tfidf,contains_emotional_emojis, 1)
# 25) hash wordvectors
emot_wordvec = numpy.append(nodoc,contains_emotional_emojis, 1)
# 26) hash docvecs
emot_docvec = numpy.append(word_vector_feature,contains_emotional_emojis, 1)



In [19]:
#########train and test######################

In [20]:
######logistic regression########

In [18]:
from numpy import mean
from numpy import std
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_predict

In [105]:
X = unigram_feature_vector#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))


scores: [0.72727273 0.81818182 0.72727273 0.90909091 0.66666667 0.81818182
 0.6969697  0.84848485 0.65625    0.78125   ]
Accuracy: 0.765 (0.079)


In [106]:
X = bigram_feature_vector#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.57575758 0.78787879 0.75757576 0.81818182 0.75757576 0.78787879
 0.75757576 0.72727273 0.71875    0.71875   ]
Accuracy: 0.741 (0.063)


In [107]:
X = trigram_feature_vector#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.54545455 0.78787879 0.75757576 0.78787879 0.6969697  0.72727273
 0.75757576 0.6969697  0.65625    0.65625   ]
Accuracy: 0.707 (0.071)


In [108]:
X = tfidf#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.72727273 0.87878788 0.78787879 0.90909091 0.72727273 0.78787879
 0.81818182 0.90909091 0.6875     0.8125    ]
Accuracy: 0.805 (0.073)


In [109]:
X = post_sent_punct#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.57575758 0.75757576 0.6969697  0.6969697  0.54545455 0.6969697
 0.54545455 0.75757576 0.53125    0.53125   ]
Accuracy: 0.634 (0.091)


In [110]:
X = pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.54545455 0.75757576 0.60606061 0.66666667 0.57575758 0.72727273
 0.6969697  0.66666667 0.53125    0.65625   ]
Accuracy: 0.643 (0.072)


In [111]:
X = nodoc#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.78787879 0.78787879 0.72727273 0.6969697  0.72727273
 0.66666667 0.78787879 0.8125     0.71875   ]
Accuracy: 0.735 (0.055)


In [112]:
X = word_vector_feature#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6969697  0.90909091 0.78787879 0.87878788 0.6969697  0.81818182
 0.84848485 0.78787879 0.8125     0.8125    ]
Accuracy: 0.805 (0.065)


In [30]:
##combinations##

In [113]:
#2
X = unigram_bi_feature_vector#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.66666667 0.84848485 0.78787879 0.84848485 0.72727273 0.81818182
 0.72727273 0.84848485 0.71875    0.8125    ]
Accuracy: 0.780 (0.062)


In [114]:
#3
X = uni_bi_tri_feat_vec#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.66666667 0.84848485 0.78787879 0.84848485 0.72727273 0.84848485
 0.72727273 0.84848485 0.71875    0.8125    ]
Accuracy: 0.783 (0.065)


In [115]:
#4
X = tfidf_bigram#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.51515152 0.81818182 0.78787879 0.84848485 0.72727273 0.78787879
 0.75757576 0.72727273 0.71875    0.71875   ]
Accuracy: 0.741 (0.086)


In [116]:
#5
X = uni_pos_new#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.72727273 0.84848485 0.75757576 0.93939394 0.72727273 0.78787879
 0.75757576 0.81818182 0.6875     0.8125    ]
Accuracy: 0.786 (0.069)


In [117]:
#6
X = uni_num#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6969697  0.84848485 0.78787879 0.93939394 0.6969697  0.78787879
 0.78787879 0.87878788 0.65625    0.78125   ]
Accuracy: 0.786 (0.083)


In [118]:
#7
X = uni_num_pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.72727273 0.87878788 0.72727273 0.93939394 0.75757576 0.78787879
 0.84848485 0.87878788 0.6875     0.8125    ]
Accuracy: 0.805 (0.077)


In [119]:
#8
X = bi_pos_new#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.60606061 0.81818182 0.87878788 0.75757576 0.75757576 0.75757576
 0.75757576 0.78787879 0.6875     0.78125   ]
Accuracy: 0.759 (0.069)


In [38]:
#9
X = bi_num#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.54545455 0.78787879 0.78787879 0.81818182 0.72727273 0.78787879
 0.78787879 0.75757576 0.6875     0.6875    ]
Accuracy: 0.738 (0.077)


In [120]:
#10
X = bi_num_pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.90909091 0.72727273 0.78787879 0.75757576 0.78787879
 0.75757576 0.75757576 0.65625    0.75      ]
Accuracy: 0.753 (0.071)


In [121]:
#11
X = tfidf_pos_new#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.60606061 0.90909091 0.78787879 0.75757576 0.72727273 0.81818182
 0.72727273 0.6969697  0.5625     0.8125    ]
Accuracy: 0.741 (0.097)


In [122]:
#12
X = tfidf_num#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.60606061 0.84848485 0.78787879 0.75757576 0.72727273 0.72727273
 0.78787879 0.84848485 0.625      0.6875    ]
Accuracy: 0.740 (0.079)


In [123]:
#13
X = tfidf_num_pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.93939394 0.75757576 0.75757576 0.75757576 0.75757576
 0.75757576 0.78787879 0.6875     0.75      ]
Accuracy: 0.759 (0.073)


In [124]:
#14
X = wordvec_pos#data
y =hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.66666667 0.78787879 0.75757576 0.72727273 0.6969697  0.72727273
 0.66666667 0.78787879 0.8125     0.71875   ]
Accuracy: 0.735 (0.048)


In [125]:
#15
X = wordvec_num#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.66666667 0.78787879 0.75757576 0.72727273 0.72727273 0.72727273
 0.6969697  0.78787879 0.84375    0.71875   ]
Accuracy: 0.744 (0.049)


In [126]:
#16
X = wordvec_num_pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.66666667 0.78787879 0.75757576 0.72727273 0.72727273 0.72727273
 0.6969697  0.78787879 0.84375    0.71875   ]
Accuracy: 0.744 (0.049)


In [127]:
#17
X = docvec_num#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.75757576 0.93939394 0.78787879 0.84848485 0.6969697  0.81818182
 0.84848485 0.75757576 0.875      0.78125   ]
Accuracy: 0.811 (0.066)


In [128]:
#18
X = docvec_pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.78787879 0.90909091 0.78787879 0.87878788 0.6969697  0.84848485
 0.84848485 0.78787879 0.8125     0.8125    ]
Accuracy: 0.817 (0.056)


In [129]:
#19
X = docvec_num_pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.75757576 0.93939394 0.78787879 0.87878788 0.6969697  0.81818182
 0.84848485 0.75757576 0.875      0.78125   ]
Accuracy: 0.814 (0.068)


In [130]:
#20
X = pos_num#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.57575758 0.84848485 0.6969697  0.6969697  0.66666667 0.78787879
 0.6969697  0.6969697  0.625      0.625     ]
Accuracy: 0.692 (0.075)


In [131]:
#21
X = all_features#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.66666667 0.81818182 0.75757576 0.72727273 0.75757576 0.72727273
 0.6969697  0.78787879 0.84375    0.71875   ]
Accuracy: 0.750 (0.052)


In [132]:
#22
X = emot_uni#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.72727273 0.81818182 0.75757576 0.90909091 0.66666667 0.81818182
 0.6969697  0.84848485 0.6875     0.78125   ]
Accuracy: 0.771 (0.074)


In [133]:
#23
X = emot_bi#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.54545455 0.78787879 0.75757576 0.81818182 0.75757576 0.81818182
 0.75757576 0.75757576 0.75       0.71875   ]
Accuracy: 0.747 (0.073)


In [134]:
#24
X = emot_tfidf#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.87878788 0.78787879 0.90909091 0.63636364 0.84848485
 0.75757576 0.90909091 0.71875    0.8125    ]
Accuracy: 0.789 (0.097)


In [135]:
#25
X = emot_wordvec#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.78787879 0.78787879 0.72727273 0.6969697  0.72727273
 0.66666667 0.78787879 0.8125     0.71875   ]
Accuracy: 0.735 (0.055)


In [136]:
#26
X = emot_docvec#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6969697  0.90909091 0.78787879 0.87878788 0.6969697  0.81818182
 0.84848485 0.78787879 0.8125     0.8125    ]
Accuracy: 0.805 (0.065)


In [137]:
#26
X = contains_emotional_emojis#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.51515152 0.78787879 0.75757576 0.84848485 0.6969697  0.75757576
 0.75757576 0.78787879 0.6875     0.65625   ]
Accuracy: 0.725 (0.088)


In [15]:
##############naive bayes##############

In [16]:
from sklearn.naive_bayes import GaussianNB

In [63]:

X = unigram_feature_vector#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.72727273 0.75757576 0.72727273 0.66666667 0.72727273
 0.63636364 0.72727273 0.71875    0.75      ]
Accuracy: 0.707 (0.042)


In [64]:

X = bigram_feature_vector#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.75757576 0.72727273 0.66666667 0.72727273 0.75757576 0.72727273
 0.75757576 0.72727273 0.78125    0.6875    ]
Accuracy: 0.732 (0.033)


In [65]:

X = trigram_feature_vector#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.57575758 0.54545455 0.42424242 0.33333333 0.51515152 0.45454545
 0.60606061 0.45454545 0.5        0.625     ]
Accuracy: 0.503 (0.085)


In [66]:

X = tfidf#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.72727273 0.78787879 0.72727273 0.66666667 0.72727273
 0.66666667 0.72727273 0.75       0.75      ]
Accuracy: 0.717 (0.044)


In [67]:
X = post_sent_punct#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.51515152 0.78787879 0.72727273 0.72727273 0.6969697  0.72727273
 0.78787879 0.78787879 0.71875    0.625     ]
Accuracy: 0.710 (0.080)


In [68]:
X = pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.51515152 0.72727273 0.78787879 0.75757576 0.6969697  0.84848485
 0.6969697  0.6969697  0.5625     0.71875   ]
Accuracy: 0.701 (0.093)


In [70]:
#WTF?
X = nodoc#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.48484848 0.21212121 0.24242424 0.24242424 0.33333333 0.24242424
 0.24242424 0.3030303  0.34375    0.34375   ]
Accuracy: 0.299 (0.078)


In [71]:

X = word_vector_feature#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.81818182 0.78787879 0.87878788 0.84848485 0.81818182 0.63636364
 0.75757576 0.90909091 0.71875    0.78125   ]
Accuracy: 0.795 (0.075)


In [72]:
##combinations##

In [73]:
#2
X = unigram_bi_feature_vector#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.78787879 0.81818182 0.78787879 0.81818182 0.78787879 0.81818182
 0.81818182 0.72727273 0.6875     0.71875   ]
Accuracy: 0.777 (0.046)


In [74]:
#3
X = uni_bi_tri_feat_vec#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.78787879 0.81818182 0.81818182 0.81818182 0.78787879 0.81818182
 0.81818182 0.72727273 0.65625    0.71875   ]
Accuracy: 0.777 (0.054)


In [75]:
#4
X = tfidf_bigram#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.75757576 0.81818182 0.81818182 0.75757576 0.87878788 0.81818182
 0.81818182 0.78787879 0.71875    0.71875   ]
Accuracy: 0.789 (0.048)


In [76]:
#5
X = uni_pos_new#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.72727273 0.75757576 0.72727273 0.66666667 0.72727273
 0.63636364 0.72727273 0.71875    0.75      ]
Accuracy: 0.707 (0.042)


In [77]:
#6
X = uni_num#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.72727273 0.75757576 0.72727273 0.66666667 0.72727273
 0.63636364 0.72727273 0.71875    0.75      ]
Accuracy: 0.707 (0.042)


In [78]:
#7
X = uni_num_pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.72727273 0.75757576 0.72727273 0.66666667 0.72727273
 0.63636364 0.72727273 0.71875    0.75      ]
Accuracy: 0.707 (0.042)


In [79]:
#8
X = bi_pos_new#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.75757576 0.72727273 0.66666667 0.72727273 0.75757576 0.72727273
 0.75757576 0.72727273 0.78125    0.6875    ]
Accuracy: 0.732 (0.033)


In [80]:
#9
X = bi_num#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.75757576 0.72727273 0.66666667 0.72727273 0.75757576 0.72727273
 0.75757576 0.72727273 0.78125    0.6875    ]
Accuracy: 0.732 (0.033)


In [81]:
#10
X = bi_num_pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.75757576 0.72727273 0.66666667 0.72727273 0.75757576 0.72727273
 0.75757576 0.72727273 0.78125    0.6875    ]
Accuracy: 0.732 (0.033)


In [82]:
#11
X = tfidf_pos_new#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.72727273 0.78787879 0.72727273 0.66666667 0.72727273
 0.66666667 0.72727273 0.75       0.75      ]
Accuracy: 0.717 (0.044)


In [83]:
#12
X = tfidf_num#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.72727273 0.78787879 0.72727273 0.66666667 0.72727273
 0.66666667 0.72727273 0.75       0.75      ]
Accuracy: 0.717 (0.044)


In [84]:
#13
X = tfidf_num_pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.72727273 0.78787879 0.72727273 0.66666667 0.72727273
 0.66666667 0.72727273 0.75       0.75      ]
Accuracy: 0.717 (0.044)


In [85]:
#14 WTF
X = wordvec_pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.48484848 0.21212121 0.24242424 0.24242424 0.33333333 0.24242424
 0.24242424 0.3030303  0.34375    0.34375   ]
Accuracy: 0.299 (0.078)


In [86]:
#15 WTF
X = wordvec_num#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.48484848 0.21212121 0.24242424 0.24242424 0.33333333 0.24242424
 0.24242424 0.3030303  0.34375    0.34375   ]
Accuracy: 0.299 (0.078)


In [88]:
#16
X = wordvec_num_pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()
# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.48484848 0.21212121 0.24242424 0.24242424 0.33333333 0.24242424
 0.24242424 0.3030303  0.34375    0.34375   ]
Accuracy: 0.299 (0.078)


In [90]:
#17
X = docvec_pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.81818182 0.78787879 0.87878788 0.84848485 0.78787879 0.63636364
 0.78787879 0.90909091 0.6875     0.8125    ]
Accuracy: 0.795 (0.078)


In [19]:
#18
X = docvec_num#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.75757576 0.78787879 0.87878788 0.84848485 0.81818182 0.63636364
 0.81818182 0.90909091 0.75       0.8125    ]
Accuracy: 0.802 (0.072)


In [94]:
#19
X = docvec_num_pos#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.75757576 0.78787879 0.87878788 0.84848485 0.81818182 0.66666667
 0.81818182 0.90909091 0.71875    0.8125    ]
Accuracy: 0.802 (0.069)


In [96]:
#20
X = pos_num#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.57575758 0.72727273 0.78787879 0.75757576 0.6969697  0.81818182
 0.75757576 0.78787879 0.625      0.6875    ]
Accuracy: 0.722 (0.073)


In [98]:
#21
X = all_features#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.81818182 0.81818182 0.84848485 0.81818182 0.78787879 0.81818182
 0.75757576 0.75757576 0.65625    0.71875   ]
Accuracy: 0.780 (0.055)


In [99]:
#21
X = emot_uni#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.72727273 0.75757576 0.72727273 0.66666667 0.72727273
 0.63636364 0.72727273 0.71875    0.75      ]
Accuracy: 0.707 (0.042)


In [100]:
#21
X = emot_bi#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.75757576 0.72727273 0.66666667 0.72727273 0.75757576 0.72727273
 0.75757576 0.72727273 0.78125    0.6875    ]
Accuracy: 0.732 (0.033)


In [101]:
#21
X = emot_tfidf#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.63636364 0.72727273 0.78787879 0.72727273 0.66666667 0.72727273
 0.66666667 0.72727273 0.75       0.75      ]
Accuracy: 0.717 (0.044)


In [102]:
#21 WTF
X = emot_wordvec#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.48484848 0.21212121 0.24242424 0.24242424 0.33333333 0.24242424
 0.24242424 0.3030303  0.34375    0.34375   ]
Accuracy: 0.299 (0.078)


In [104]:
#21
X = emot_docvec#data
y = hash_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.75757576 0.78787879 0.87878788 0.87878788 0.75757576 0.63636364
 0.75757576 0.93939394 0.75       0.78125   ]
Accuracy: 0.793 (0.082)
